In [4]:
# 通过使用深度学习框架来简洁地实现线性回归模型生成数据集
import numpy as np
import torch 
from torch.utils import data
from d2l import torch as d2l

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

In [5]:
# 调用框架中现有的API来读取数据
"""
===========构造一个PyTorch数据迭代器=========
data_arrays 包含特征和标签的元组，如（features, labels)
is_train 是否为训练模式（训练时打乱数据）
"""
def load_array(data_arrays, batch_size, is_train=True):   
    # 创建数据集
    dataset = data.TensorDataset(*data_arrays)
    # 创建数据加载器
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

    # shuffle=True 打乱样本顺序


batch_size = 10
data_iter = load_array((features, labels), batch_size)

# 获取一个批次的数据
# iterator = iter(data_iter) 获取数据迭代器
# batch = next(iterator) 获取第一个批次
next(iter(data_iter))

[tensor([[-0.9878,  1.9181],
         [-0.9853, -0.2909],
         [-0.3387,  0.5745],
         [ 0.8522, -0.4034],
         [ 0.4877,  1.4810],
         [ 0.7356,  0.5072],
         [ 0.6862,  0.4487],
         [ 0.1075, -1.6205],
         [ 0.0781, -0.6835],
         [-3.1517,  0.2339]]),
 tensor([[-4.2987],
         [ 3.2191],
         [ 1.5634],
         [ 7.2769],
         [ 0.1293],
         [ 3.9406],
         [ 4.0591],
         [ 9.9056],
         [ 6.6901],
         [-2.9105]])]

In [11]:
# 使用框架的预定义好的层
from torch import nn  # nn 是神经网络的缩写

# 创建一个顺序神经网络
# nn.Sequential按顺序执行
# nn.Linear(2, 1)一个全连接层，输入特征数量2，输出特征数量1
net = nn.Sequential(nn.Linear(2, 1))

In [12]:
# net[0]  访问 nn.Sequential 中的第一个层，即 nn.Linear(2, 1)
# 在原地使用正态分布采样数据
net[0].weight.data.normal_(0, 0.01) # 初始化权重 w


net[0].bias.data.fill_(0) # 初始化偏置 b

tensor([0.])

In [13]:
# 计算均方误差使用的是MSELoss类，也称平方L2范数
# 创建一个均方误差(MSE)损失函数
loss = nn.MSELoss()

In [14]:
# 实例化SGD（批量随机梯度下降）实例
# optim 是优化器（optimizer）模块
# SGD 是最基本、最常用的优化算法之一
# net.parameters()  获取神经网络中所有需要训练的参数
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

In [22]:
# 训练过程
num_epochs = 3 
for epoch in range(num_epochs):
    for X, y in data_iter: # 自动迭代所有批次
        # X:当前批次的特征[batch_size, n_features]
        # y:当前批次的标签[batch_size]

        l = loss(net(X), y)
        trainer.zero_grad()
        l.backward()
        trainer.step()

    # 每个epoch结束后，在整个数据集上计算损失
    with torch.no_grad():  # 禁用梯度计算，节省内存
        total_loss = loss(net(features), labels)
        print(f'epoch {epoch + 1}, loss {total_loss.item():f}')
        # total_loss.item():f 固定小数格式，默认显示6位小数
        # total_loss.item():.4f 保留4位小数

epoch 1, loss 0.000109
epoch 2, loss 0.000108
epoch 3, loss 0.000107
